In [1]:
import pandas as pd
import numpy as np
import dask.distributed

import tsfresh
import tsfresh.utilities.distribution

#### Load raw data from `processed_raw_data.parquet`

This data was generated in `002_process-raw-data.ipynb`.

In [2]:
df = pd.read_parquet('processed_raw_data.parquet')

# Put `t_ms_rounded` and `segment_id` back in dataframe
df = df.reset_index(['t_ms_rounded', 'segment_id'])

# Drop any rows with NaNs (only about 3%)
df.dropna(how='any', inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3216364 entries, (1600, 'A', 'walking', 1) to (1650, 'S', 'folding', 12)
Data columns (total 14 columns):
 #   Column         Dtype  
---  ------         -----  
 0   t_ms_rounded   int64  
 1   segment_id     int64  
 2   accel_phone_x  float64
 3   accel_phone_y  float64
 4   accel_phone_z  float64
 5   accel_watch_x  float64
 6   accel_watch_y  float64
 7   accel_watch_z  float64
 8   gyro_phone_x   float64
 9   gyro_phone_y   float64
 10  gyro_phone_z   float64
 11  gyro_watch_x   float64
 12  gyro_watch_y   float64
 13  gyro_watch_z   float64
dtypes: float64(12), int64(2)
memory usage: 355.8+ MB


In [3]:
df.head(9)

t_ms_rounded  \
subject_id activity_id activity_name segment_sequence_number                 
1600       A           walking       1                                   0   
                                     1                                  50   
                                     1                                 100   
                                     1                                 150   
                                     1                                 200   
                                     1                                 250   
                                     1                                 300   
                                     1                                 350   
                                     1                                 400   

                                                              segment_id  \
subject_id activity_id activity_name segment_sequence_number               
1600       A           walking       1                                 1   
                                     1                                 1   
                                     1                                 1   
                                     1                                 1   
                                     1                                 1   
                                     1                                 1   
                                     1                                 1   
                                     1                                 1   
                                     1                                 1   

                                                              accel_phone_x  \
subject_id activity_id activity_name segment_sequence_number                  
1600       A           walking       1                            -0.364761   
                                     1                            -0.879730   
                                     1                             2.001495   
                                     1                             0.450623   
                                     1                            -2.164352   
                                     1                            -4.332779   
                                     1                            -0.319443   
                                     1                             1.566452   
                                     1                            -0.323746   

                                                              accel_phone_y  \
subject_id activity_id activity_name segment_sequence_number                  
1600       A           walking       1                             8.793503   
                                     1                             9.768784   
                                     1                            11.109070   
                                     1                            12.651642   
                                     1                            13.928436   
                                     1                            13.361191   
                                     1                            13.318359   
                                     1                             9.515274   
                                     1                             5.262665   

                                                              accel_phone_z  \
subject_id activity_id activity_name segment_sequence_number                  
1600       A           walking       1                             1.055084   
                                     1                             1.016998   
                                     1                             2.619156   
                                     1                             0.184555   
                                     1                            -4.422485   
                                     1                            -0.7188

#### Use `tsfresh` to calculate features

In [18]:
%%time

features_df = tsfresh.extract_features(
    timeseries_container = df,
    # y = y,
    column_id = 'segment_id',
    column_sort = 't_ms_rounded',
)

features_df.info()

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 60/60 [33:29<00:00, 33.49s/it]
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10809 entries, 1 to 11138
Columns: 9444 entries, accel_phone_x__variance_larger_than_standard_deviation to gyro_watch_z__matrix_profile__feature_"75"__threshold_0.98
dtypes: float64(9444)
memory usage: 778.9 MB
Wall time: 35min 38s


distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)


In [19]:
features_df.sample(7)

,accel_phone_x__variance_larger_than_standard_deviation,accel_phone_x__has_duplicate_max,accel_phone_x__has_duplicate_min,accel_phone_x__has_duplicate,accel_phone_x__sum_values,accel_phone_x__abs_energy,accel_phone_x__mean_abs_change,accel_phone_x__mean_change,accel_phone_x__mean_second_derivative_central,accel_phone_x__median,...,gyro_watch_z__permutation_entropy__dimension_5__tau_1,gyro_watch_z__permutation_entropy__dimension_6__tau_1,gyro_watch_z__permutation_entropy__dimension_7__tau_1,gyro_watch_z__query_similarity_count__query_None__threshold_0.0,"gyro_watch_z__matrix_profile__feature_""min""__threshold_0.98","gyro_watch_z__matrix_profile__feature_""max""__threshold_0.98","gyro_watch_z__matrix_profile__feature_""mean""__threshold_0.98","gyro_watch_z__matrix_profile__feature_""median""__threshold_0.98","gyro_watch_z__matrix_profile__feature_""25""__threshold_0.98","gyro_watch_z__matrix_profile__feature_""75""__threshold_0.98"
620,0.0,0.0,0.0,1.0,-600.953201,1218.747082,0.067230,0.000138,-0.000168,-2.036819,...,4.493948,5.372540,5.596045,NaN,0.669328,2.549272,1.785258,1.864947,1.583503,2.071668
202,0.0,0.0,0.0,1.0,1115.446351,4194.106098,0.111031,0.000288,0.000142,3.769562,...,4.236267,5.039053,5.409180,NaN,0.897949,3.850784,1.853358,1.771150,1.550873,2.090641
2636,1.0,0.0,0.0,0.0,-877.759109,6328.696055,2.414102,-0.019140,0.002170,-2.634621,...,3.137897,3.823181,4.396022,NaN,0.926674,3.640245,1.855424,1.636038,1.154958,2.586669
4957,1.0,0.0,0.0,0.0,920.624921,3473.970104,0.895322,-0.001396,0.000544,3.217819,...,3.036091,3.819922,4.469277,NaN,0.904836,3.480074,1.730432,1.689701,1.331974,2.099079
7787,0.0,0.0,0.0,1.0,-2075.406020,14504.787405,0.040507,-0.000189,0.000147,-6.980301,...,4.009356,4.893539,5.363539,NaN,0.757048,3.361426,1.720681,1.689373,1.306054,2.155172
5184,0.0,0.0,0.0,0.0,-218.935897,307.908402,0.300922,-0.007367,-0.000852,-0.855362,...,2.581510,3.172045,3.700343,NaN,1.540977,6.692903,2.871212,2.720460,1.801677,3.316842
6495,0.0,0.0,0.0,1.0,-1127.043410,4282.727660,0.026700,-0.000018,-0.000014,-3.860214,...,4.016120,4.779529,5.208506,NaN,1.437791,6.378184,3.665348,3.380794,2.576935,4.912287


#### Write `features_df` to disk

In [20]:
features_df.to_parquet('tsfresh_features.parquet', index=True)

distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
